In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import datetime
import h3
import folium
from folium.plugins import MarkerCluster
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

2025-05-12 16:49:35.567418: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/tecorb/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Load data
csv_path = 'yellow_tripdata_2016-01.csv'
df = pd.read_csv(csv_path, parse_dates=['tpep_pickup_datetime'])

In [4]:
# Load data
csv_path2 = 'yellow_tripdata_2016-02.csv'
df2 = pd.read_csv(csv_path2, parse_dates=['tpep_pickup_datetime'])

In [5]:
import pandas as pd

# Merge both datasets by stacking them vertically
df_merged = pd.concat([df, df2], ignore_index=True)

# Optional: sort by pickup datetime if needed
df_merged.sort_values(by='tpep_pickup_datetime', inplace=True)

In [6]:
df_merged.head(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-01-01,2016-01-01 00:00:00,2,1.1,-73.990372,40.734695,1,N,-73.981842,40.732407,2,7.5,0.5,0.5,0.0,0.0,0.3,8.8
1,2,2016-01-01,2016-01-01 00:00:00,5,4.9,-73.980782,40.729912,1,N,-73.944473,40.716679,1,18.0,0.5,0.5,0.0,0.0,0.3,19.3


In [7]:
# Filter necessary columns
df_merged = df_merged[['tpep_pickup_datetime', 'pickup_latitude', 'pickup_longitude', 'passenger_count']]

# Drop rows with missing values
df_merged = df_merged.dropna(subset=['pickup_latitude', 'pickup_longitude'])

# Filter out invalid coordinates
df_merged = df_merged[(df_merged['pickup_latitude'] > 40.5) & (df_merged['pickup_latitude'] < 41.0) &
        (df_merged['pickup_longitude'] > -74.05) & (df_merged['pickup_longitude'] < -73.7)]

print(f"Dataset shape after preprocessing: {df_merged.shape}")

Dataset shape after preprocessing: (21933115, 4)


In [8]:
df_merged

,tpep_pickup_datetime,pickup_latitude,pickup_longitude,passenger_count
0,2016-01-01 00:00:00,40.734695,-73.990372,2
1,2016-01-01 00:00:00,40.729912,-73.980782,5
2,2016-01-01 00:00:00,40.679565,-73.984550,1
3,2016-01-01 00:00:00,40.718990,-73.993469,1
4,2016-01-01 00:00:00,40.781330,-73.960625,3
...,...,...,...,...
20800027,2016-02-29 23:59:57,40.759048,-73.985748,1
20800029,2016-02-29 23:59:58,40.773479,-73.960068,1
20800031,2016-02-29 23:59:58,40.773155,-73.954674,5
20800028,2016-02-29 23:59:58,40.779552,-73.955589,1


In [9]:
import pandas as pd
import h3
import folium
import branca.colormap as cm
from datetime import datetime
import os

# Weekday name mapping
weekday_map = {
    0: 'Monday', 1: 'Tuesday', 2: 'Wednesday',
    3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'
}

def preprocess_and_filter(df, start_date, weekday, start_hour, end_hour, num_days):
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    start_date = pd.to_datetime(start_date)
    end_date = start_date + pd.Timedelta(days=num_days)

    df = df[(df['tpep_pickup_datetime'] >= start_date) & (df['tpep_pickup_datetime'] < end_date)]
    df = df[df['tpep_pickup_datetime'].dt.weekday == weekday]

    pickup_time = df['tpep_pickup_datetime'].dt.hour + df['tpep_pickup_datetime'].dt.minute / 60.0
    df = df[(pickup_time >= start_hour) & (pickup_time < end_hour)]

    return df[['tpep_pickup_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude']]

def add_h3_and_group(df, resolution=8):
    coords = list(zip(df['pickup_latitude'], df['pickup_longitude']))
    df['h3_index'] = [h3.geo_to_h3(lat, lng, resolution) for lat, lng in coords]
    df['pickup_date'] = df['tpep_pickup_datetime'].dt.date
    return df[['tpep_pickup_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'h3_index', 'pickup_date']]

def plot_hex_map(df, filename):
    df['hex_boundary'] = df['h3_index'].apply(lambda h: [[lat, lng] for lat, lng in h3.h3_to_geo_boundary(h)])
    df['center_lat'] = df['h3_index'].apply(lambda h: h3.h3_to_geo(h)[0])
    df['center_lng'] = df['h3_index'].apply(lambda h: h3.h3_to_geo(h)[1])

    map_center = [df['center_lat'].mean(), df['center_lng'].mean()]
    m = folium.Map(location=map_center, zoom_start=12)

    total_passenger_by_hex = df.groupby('h3_index')['passenger_count'].sum().reset_index()
    max_pass = total_passenger_by_hex['passenger_count'].max()
    min_pass = total_passenger_by_hex['passenger_count'].min()

    colormap = cm.LinearColormap(colors=['green', 'yellow', 'red'], vmin=min_pass, vmax=max_pass,
                                 caption='Total Passenger Count (by Hex)')

    for _, row in total_passenger_by_hex.iterrows():
        h3_index = row['h3_index']
        total_passengers = row['passenger_count']
        boundary = df[df['h3_index'] == h3_index]['hex_boundary'].iloc[0]
        fill_color = colormap(total_passengers)

        popup_html = f"<b>H3 Index:</b> {h3_index}<br><b>Total Passengers:</b> {total_passengers}"

        folium.Polygon(
            locations=boundary,
            color='black',
            weight=1,
            fill=True,
            fill_color=fill_color,
            fill_opacity=0.6,
            popup=folium.Popup(popup_html, max_width=300)
        ).add_to(m)

    colormap.add_to(m)
    
    html_path = os.path.join("/Users/tecorb/Downloads/myproject/Datasets_Map_html", filename + ".html")
    m.save(html_path)
    print(f"Map saved as {html_path}")

def format_hour_label(hour):
    suffix = 'AM' if hour < 12 else 'PM'
    return f"{int(hour)}_{suffix}"

def get_filename(weekday, start_hour, end_hour):
    day_name = weekday_map.get(weekday, f'Day{weekday}')
    start_label = format_hour_label(start_hour)
    end_label = format_hour_label(end_hour)
    return f"{day_name}_{start_label}_to_{end_label}_data"

def main_pipeline(df_merged):
    while True:
        try:
            weekday = int(input("Enter weekday (0=Monday, ..., 6=Sunday): "))
            start_hour = float(input("Enter start hour (e.g., 0, 13, 17.25): "))
            end_hour = float(input("Enter end hour (e.g., 1, 14, 18.25): "))
            num_days = int(input("Enter number of days to include: "))
            start_date = input("Enter start date (YYYY-MM-DD): ")
        except Exception as e:
            print(f"Invalid input: {e}")
            continue

        print("Processing data...")
        filtered_df = preprocess_and_filter(df_merged, start_date, weekday, start_hour, end_hour, num_days)
        if filtered_df.empty:
            print("No data found for given filters.")
            continue

        final_df = add_h3_and_group(filtered_df)
        filename = get_filename(weekday, start_hour, end_hour)

        # Save CSV
        csv_path = os.path.join("/Users/tecorb/Downloads/myproject/datasets", filename + ".csv")
        final_df.to_csv(csv_path, index=False)
        print(f"Data saved to {csv_path}")

        # Save HTML map
        plot_hex_map(final_df, filename)

        cont = input("Do you want to extract more data? (yes/no): ").lower()
        if cont != 'yes':
            print("Pipeline finished.")
            break


In [10]:
main_pipeline(df_merged)

Processing data...
Data saved to /Users/tecorb/Downloads/myproject/datasets/Sunday_3_AM_to_4_AM_data.csv
Map saved as /Users/tecorb/Downloads/myproject/Datasets_Map_html/Sunday_3_AM_to_4_AM_data.html
Processing data...
Data saved to /Users/tecorb/Downloads/myproject/datasets/Sunday_15_PM_to_16_PM_data.csv
Map saved as /Users/tecorb/Downloads/myproject/Datasets_Map_html/Sunday_15_PM_to_16_PM_data.html
Pipeline finished.
